In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!python --version

In [ ]:
print(tf.__version__)
print(keras.__version__)

In [ ]:
train_path = "../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
test_path = "../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test"

In [ ]:
train_batches = ImageDataGenerator(validation_split=0.1) \
                .flow_from_directory(directory=train_path,  
                                     classes = ['NonDemented', 'VeryMildDemented', 
                                                'MildDemented', 'ModerateDemented'], 
                                     subset='training',
                                     batch_size=10)

validation_batches = ImageDataGenerator(validation_split=0.1) \
                     .flow_from_directory(directory=train_path, 
                                          classes = ['NonDemented', 'VeryMildDemented', 
                                                     'MildDemented', 'ModerateDemented'], 
                                          subset='validation',
                                          batch_size=10)
                
test_batches = ImageDataGenerator() \
                    .flow_from_directory(directory=test_path, 
                                         classes = ['NonDemented', 'VeryMildDemented', 
                                                    'MildDemented', 'ModerateDemented'], 
                                         batch_size=10, 
                                         shuffle=False)

In [ ]:
data = {'NonDemented':      0, 
        'VeryMildDemented': 0, 
        'MildDemented':     0,
        'ModerateDemented': 0}

for cls in os.listdir(train_path):
    for img in os.listdir(train_path + '/' + cls):
        data[cls] = data[cls] + 1

keys = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (10, 5))
 
plt.bar(keys, values, color=(0.2, 0.4, 0.6, 0.6), width = 0.4)

Data Preprocessing

In [ ]:
IMG_SIZE = 224
n_classes = 4

model = Sequential([layers.Resizing(IMG_SIZE, IMG_SIZE),
                    layers.Rescaling(1./255),
                    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', 
                                  padding = 'same', input_shape=(IMG_SIZE,IMG_SIZE,3)),
                    layers.BatchNormalization(),
                    layers.MaxPool2D(pool_size=(2, 2)),
                    layers.Conv2D(filters=64, kernel_size=(3, 3), 
                                  activation='relu', padding = 'same'),
                    layers.BatchNormalization(),
                    layers.MaxPool2D(pool_size=(2, 2)),
                    layers.Conv2D(filters=128, kernel_size=(3, 3), 
                                  activation='relu', padding = 'same'),
                    layers.BatchNormalization(),
                    layers.MaxPool2D(pool_size=(2, 2)),
                    layers.Conv2D(filters=256, kernel_size=(3, 3), 
                                  activation='relu', padding = 'same'),
                    layers.BatchNormalization(),
                    layers.MaxPool2D(pool_size=(2, 2)),
                    layers.Flatten(),
                    layers.Dense(300, activation='relu'),
                    layers.Dense(150, activation='relu'),
                    layers.Dropout(0.25),
                    layers.Dense(n_classes, activation='softmax')])


In [ ]:
METRICS = [keras.metrics.CategoricalAccuracy(name='accuracy'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.AUC(name='auc')]

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=METRICS)

In [ ]:
EPOCHS = 50
BATCH_SIZE = 8

In [ ]:
history = model.fit(x=train_batches,
                    validation_data=validation_batches,
                    steps_per_epoch=len(train_batches),
                    validation_steps=len(validation_batches),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE, 
                    verbose=2)

In [ ]:
model.summary()

In [ ]:
def plot_loss(history, n):
    plt.semilogy(history.epoch, history.history['loss'], label='Train')
    plt.semilogy(history.epoch, history.history['val_loss'], label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

In [ ]:
plot_loss(history, 0)

In [ ]:
results = model.evaluate(test_batches, verbose=0)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)